In [ ]:
#!pip install placekey
from placekey.api import PlacekeyAPI
from ast import literal_eval
import snowflake.connector
from snowflake.connector import cursor
import usaddress
import re
import pandas as pd
from helper_functions import *
import config_final

In [ ]:
ctx = snowflake.connector.connect(
    user=config_final.user,
    password=config_final.password,
    account=config_final.account
)
cursor = ctx.cursor()

sql_MLS = """SELECT (CASE WHEN CC_PROPERTY_ADDR_DISPLAY_1 IS NULL THEN ADDRESS ELSE CC_PROPERTY_ADDR_DISPLAY_1 END) AS MLS_ADDRESS,
                    OWNER_FULL_NAME AS NAME,
                    CC_PROPERTY_ADDR_POSTAL_CODE AS POSTALCODE
            FROM ATTOM.MLS
            WHERE CC_PROPERTY_ADDR_CITY = 'phoenix' and CC_PROPERTY_ADDR_STATE='az' limit 15000"""
df_MLS = pd.read_sql(sql_MLS, con=ctx)
sql_DATATREE = """SELECT SITUSFULLSTREETADDRESS AS DATATREE_ADDRESS,
SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,SITUSUNITNBR,SITUSLATITUDE,SITUSLONGITUDE,
                         OWNERNAME1FULL,
                         OWNERNAME2FULL,
                         SITUSZIP5
                         
                  FROM DATATREE.ASSESSOR
                  WHERE SITUSCITY = 'phoenix'and SITUSSTATE='az'limit 15000 """
df_DATATREE = pd.read_sql(sql_DATATREE, con=ctx)


## completely match 

In [ ]:
# Drop NaN
df_MLS.dropna(subset=['MLS_ADDRESS'], inplace=True)
df_DATATREE.dropna(subset=['DATATREE_ADDRESS'], inplace=True)
# Merge the data
df_merge_node1 = pd.merge(df_DATATREE,df_MLS, how = 'left', right_on = ['MLS_ADDRESS', 'POSTALCODE'], 
                          left_on = ['DATATREE_ADDRESS', 'SITUSZIP5'])
# Compute ratio
first_node_match = len(df_merge_node1[df_merge_node1['MLS_ADDRESS'].notnull()]['MLS_ADDRESS'].unique())
base = len(df_DATATREE['DATATREE_ADDRESS'].unique())
first_node_match_ratio = first_node_match/base
print(first_node_match_ratio)

MLS_matched_address = []
DATATREE_matched_address = []

first_node_match_list = list(df_merge_node1[df_merge_node1['MLS_ADDRESS'].notnull()]['MLS_ADDRESS'].unique())
MLS_matched_address.extend(first_node_match_list)
DATATREE_matched_address.extend(first_node_match_list)
print(len(MLS_matched_address))
print(len(DATATREE_matched_address))
df_first_node = df_merge_node1[df_merge_node1['DATATREE_ADDRESS'].isin(DATATREE_matched_address)]

0.0320149356935417
463
463


## transform and convert address 

In [ ]:
df_MLS = clean_special_char(df_MLS,'MLS_ADDRESS')
df_DATATREE = clean_special_char(df_DATATREE,'DATATREE_ADDRESS')

In [ ]:
df1_MLS = df_MLS[:15000] # only take a sample with the first 15000 rows
df1_MLS['MLS_ADDRESS_Split'] = df1_MLS.apply(lambda df1_MLS : get_splited_addr(df1_MLS,'MLS_ADDRESS'), axis = 1)
df1_DATATREE = df_DATATREE[:15000]
df1_DATATREE['DATATREE_ADDRESS_Split'] = df1_DATATREE.apply(lambda df1_DATATREE : get_splited_addr(df1_DATATREE,'DATATREE_ADDRESS'), axis = 1)


In [ ]:
# combine with original dataset
# DT_ADDRESS_final.SITUSUNITTYPE.unique()
# DT_ADDRESS_final.SITUSSTREET.unique()

MLS_ADDRESS_final = get_comb_addr(df1_MLS,'MLS_ADDRESS_Split')
DT_ADDRESS_final = get_comb_addr(df1_DATATREE,'DATATREE_ADDRESS_Split')

# convert address format (direction, abbreviation)
MLS_ADDRESS_final,DT_ADDRESS_final=get_str_replace(MLS_ADDRESS_final,DT_ADDRESS_final)

In [ ]:
# pick some test dataset
# DT_ADDRESS_final[(DT_ADDRESS_final['SITUSUNITTYPE']=='unit')&(DT_ADDRESS_final['SITUSSTREET']=='tatum')]
test1 = DT_ADDRESS_final[(DT_ADDRESS_final['SITUSUNITTYPE']=='unit')&(DT_ADDRESS_final['SITUSSTREET']=='van buren')]
test1 = test1.drop_duplicates(subset='DATATREE_ADDRESS')

test2 = DT_ADDRESS_final[(DT_ADDRESS_final['SITUSUNITTYPE']=='apt')&(DT_ADDRESS_final['SITUSSTREET']=='monroe')]
test2 = test2.drop_duplicates(subset='DATATREE_ADDRESS')

In [ ]:
test1

,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,SITUSUNITNBR,SITUSLATITUDE,SITUSLONGITUDE,...,StreetNamePreDirectional,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,StreetNamePreType,PlaceName,StateName,AddressNumberSuffix
1156,5401 e van buren st unit 1084,5401,e,van buren,st,None,unit,1084,33.449058,-111.964291,...,e,van buren,st,unit,1084,NaN,NaN,NaN,NaN,NaN
2327,5401 e van buren st unit 3109,5401,e,van buren,st,None,unit,3109,33.449058,-111.964291,...,e,van buren,st,unit,3109,NaN,NaN,NaN,NaN,NaN
2354,5401 e van buren st unit 2101,5401,e,van buren,st,None,unit,2101,33.449058,-111.964291,...,e,van buren,st,unit,2101,NaN,NaN,NaN,NaN,NaN
2373,5401 e van buren st unit 3042,5401,e,van buren,st,None,unit,3042,33.449058,-111.964291,...,e,van buren,st,unit,3042,NaN,NaN,NaN,NaN,NaN
2424,5302 e van buren st unit 1054,5302,e,van buren,st,None,unit,1054,33.455038,-112.175458,...,e,van buren,st,unit,1054,NaN,NaN,NaN,NaN,NaN
2740,5345 e van buren st unit 325,5345,e,van buren,st,None,unit,325,33.449607,-111.965920,...,e,van buren,st,unit,325,NaN,NaN,NaN,NaN,NaN
4071,5401 e van buren st unit 2120,5401,e,van buren,st,None,unit,2120,33.449058,-111.964291,...,e,van buren,st,unit,2120,NaN,NaN,NaN,NaN,NaN
4077,5401 e van buren st unit 3022,5401,e,van buren,st,None,unit,3022,33.449058,-111.964291,...,e,van buren,st,unit,3022,NaN,NaN,NaN,NaN,NaN
4518,5401 e van buren st unit 1109,5401,e,van buren,st,None,unit,1109,33.449058,-111.964291,...,e,van buren,st,unit,1109,NaN,NaN,NaN,NaN,NaN
5277,5401 e van buren st unit 3079,5401,e,van buren,st,None,unit,3079,33.449058,-111.964291,...,e,van buren,st,unit,3079,NaN,NaN,NaN,NaN,NaN


In [ ]:
# test2

In [ ]:
test3 = DT_ADDRESS_final[(DT_ADDRESS_final['SITUSUNITTYPE']=='#')&(DT_ADDRESS_final['SITUSSTREET']=='grandview')]
# test3
# DT_ADDRESS_final[(DT_ADDRESS_final['SITUSUNITTYPE']=='#')].head(70)


## try with placekey.io 

In [ ]:
from placekey.api import PlacekeyAPI
placekey_api_key = "n1w1KOroOdSz3BVMMlg8IM4PuflcVP4V" # fill this in with your personal API key (do not share publicly)
pk_api = PlacekeyAPI(placekey_api_key)
column_map = {"DATATREE_ADDRESS" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")


## DEAL WITH ASSESSOR TABLE

### Question1: Does the unit signal matter?
#### Note1: we want to have certain unit signal before the unit number 
reasons:
1. to get rid of having the same key with different unit numbers
2. make sure the key is correct - we don't want to lose original information!

In [ ]:
# without unit signal
stad_df = get_standard_df(test3,column_map,"US", "phoenix", "AZ")
df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
pd.concat([df_placekey,test3.reset_index()],axis=1)

# notice: query_id #4,5,6 have the same placekey in spite of different unit numbers

2022-04-06 16:10:27,928	INFO	Processed 8 items
2022-04-06 16:10:27,931	INFO	Done


,query_id,placekey,index,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,...,StreetNamePreDirectional,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,StreetNamePreType,PlaceName,StateName,AddressNumberSuffix
0,1,24s@5zb-wx6-5zz,5358,3033 e grandview rd 1,3033,e,grandview,rd,None,#,...,e,grandview,rd,NaN,1,NaN,NaN,NaN,NaN,NaN
1,2,24s@5zb-wx6-5zz,5359,3033 e grandview rd 3,3033,e,grandview,rd,None,#,...,e,grandview,rd,NaN,3,NaN,NaN,NaN,NaN,NaN
2,3,22q@5zb-wx6-6hq,5364,3039 e grandview rd 1,3039,e,grandview,rd,None,#,...,e,grandview,rd,NaN,1,NaN,NaN,NaN,NaN,NaN
3,4,22q@5zb-wx6-6hq,5365,3039 e grandview rd 2,3039,e,grandview,rd,None,#,...,e,grandview,rd,NaN,2,NaN,NaN,NaN,NaN,NaN
4,5,22q@5zb-wx6-6hq,5366,3039 e grandview rd 4,3039,e,grandview,rd,None,#,...,e,grandview,rd,NaN,4,NaN,NaN,NaN,NaN,NaN
5,6,22p@5zb-wx6-6hq,5367,3045 e grandview rd 2,3045,e,grandview,rd,None,#,...,e,grandview,rd,NaN,2,NaN,NaN,NaN,NaN,NaN
6,7,22p@5zb-wx6-6hq,5368,3045 e grandview rd 3,3045,e,grandview,rd,None,#,...,e,grandview,rd,NaN,3,NaN,NaN,NaN,NaN,NaN
7,8,22p@5zb-wx6-6hq,5369,3045 e grandview rd 4,3045,e,grandview,rd,None,#,...,e,grandview,rd,NaN,4,NaN,NaN,NaN,NaN,NaN


In [ ]:
test3['ColumnA'] = test3[test3.columns[1:8]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")

stad_df = get_standard_df(test3,column_map,"US", "phoenix", "AZ")
df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
pd.concat([df_placekey,test3.reset_index()],axis=1).to_csv('expe.csv')
# this time they work fine, but a new issue is generated --> the "where" part in #1,#2 changed! 

2022-04-07 11:38:47,263	INFO	Processed 8 items
2022-04-07 11:38:47,265	INFO	Done


In [ ]:
# another example with unit signal
# example with "monroe"
column_map = {"DATATREE_ADDRESS" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")

stad_df = get_standard_df(test2,column_map,"US", "phoenix", "AZ")
df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
pd.concat([df_placekey,test2.reset_index()],axis=1)

2022-04-07 10:53:17,385	INFO	Processed 5 items
2022-04-07 10:53:17,387	INFO	Done


,query_id,placekey,index,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,...,StreetNamePreDirectional,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,StreetNamePreType,PlaceName,StateName,AddressNumberSuffix
0,1,233@5zb-xdy-ndv,2583,44 w monroe st apt 1009,44,w,monroe,st,None,apt,...,w,monroe,st,apt,1009,NaN,NaN,NaN,NaN,NaN
1,2,222@5zb-xdy-ndv,6623,44 w monroe st apt 1505,44,w,monroe,st,None,apt,...,w,monroe,st,apt,1505,NaN,NaN,NaN,NaN,NaN
2,3,24t@5zb-xdy-ndv,6666,44 w monroe st apt 1208,44,w,monroe,st,None,apt,...,w,monroe,st,apt,1208,NaN,NaN,NaN,NaN,NaN
3,4,26v@5zb-xdy-ndv,11265,44 w monroe st apt 2803,44,w,monroe,st,None,apt,...,w,monroe,st,apt,2803,NaN,NaN,NaN,NaN,NaN
4,5,25m@5zb-xdy-ndv,13476,44 w monroe st apt 1408,44,w,monroe,st,None,apt,...,w,monroe,st,apt,1408,NaN,NaN,NaN,NaN,NaN


#### conclusioin: 
1. for Assessor table having detailed address info, we use original address (or the new combined column we generated by ourselves) to match, bc the splited info without "#" is not accurate
2. api requires an occuring unit signal before unit number 

### Question2: What's the tolerance for matching accuracy? -- in terms of the "where" part
#### Note2: we need to figure out what's wrong with the different "where" parts given the same addresses with the only difference in their unit info
#### description of the newly arised issue : 
1. the "where" part in #1,#2 changed! -- see previous output or the following cell\
* notes: \
when the "where" parts have 7 shared prefix => the max distance is 1,176m\
when they have 6 shared prefix => maximal distance is 8,227m

In [ ]:
# this cell is the same as the previous one
test3['ColumnA'] = test3[test3.columns[1:8]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
# column_map = {"DATATREE_ADDRESS" : "street_address", "SITUSZIP5": "postal_code"} # "what" fails, "where" works
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"} # "what" works, "where" fails
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")

stad_df = get_standard_df(test3,column_map,"US", "phoenix", "AZ")
df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
pd.concat([df_placekey,test3.reset_index()],axis=1)
# .to_csv('az.csv')
# this time they work fine, but a new issue is generated --> the "where" part in #1,#2 changed! 
# but this is not always the case -> for index #2,3,4, the placekey works!


2022-04-06 23:21:14,545	INFO	Processed 8 items
2022-04-06 23:21:14,549	INFO	Done


,query_id,placekey,index,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,...,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,StreetNamePreType,PlaceName,StateName,AddressNumberSuffix,ColumnA
0,1,24c@5zb-wx6-5zz,5358,3033 e grandview rd 1,3033,e,grandview,rd,None,#,...,grandview,rd,NaN,1,NaN,NaN,NaN,NaN,NaN,3033 e grandview rd # 1
1,2,22g@5zb-wx6-6hq,5359,3033 e grandview rd 3,3033,e,grandview,rd,None,#,...,grandview,rd,NaN,3,NaN,NaN,NaN,NaN,NaN,3033 e grandview rd # 3
2,3,228@5zb-wx6-6hq,5364,3039 e grandview rd 1,3039,e,grandview,rd,None,#,...,grandview,rd,NaN,1,NaN,NaN,NaN,NaN,NaN,3039 e grandview rd # 1
3,4,229@5zb-wx6-6hq,5365,3039 e grandview rd 2,3039,e,grandview,rd,None,#,...,grandview,rd,NaN,2,NaN,NaN,NaN,NaN,NaN,3039 e grandview rd # 2
4,5,224@5zb-wx6-6hq,5366,3039 e grandview rd 4,3039,e,grandview,rd,None,#,...,grandview,rd,NaN,4,NaN,NaN,NaN,NaN,NaN,3039 e grandview rd # 4
5,6,22f@5zb-wx6-6hq,5367,3045 e grandview rd 2,3045,e,grandview,rd,None,#,...,grandview,rd,NaN,2,NaN,NaN,NaN,NaN,NaN,3045 e grandview rd # 2
6,7,225@5zb-wx6-6hq,5368,3045 e grandview rd 3,3045,e,grandview,rd,None,#,...,grandview,rd,NaN,3,NaN,NaN,NaN,NaN,NaN,3045 e grandview rd # 3
7,8,22b@5zb-wx6-6hq,5369,3045 e grandview rd 4,3045,e,grandview,rd,None,#,...,grandview,rd,NaN,4,NaN,NaN,NaN,NaN,NaN,3045 e grandview rd # 4


In [ ]:
# another example with unit signal & the new issued mentioned above
# example with "van buren"
column_map = {"DATATREE_ADDRESS" : "street_address", "SITUSZIP5": "postal_code"} # don't need the combined column bc the original address has a unit signal
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")

stad_df = get_standard_df(test1,column_map,"US", "phoenix", "AZ")
df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
pd.concat([df_placekey,test1.reset_index()],axis=1).head(8)

# new issue: for index #1,#2,#5, the "where" part also changes

2022-04-06 16:37:29,778	INFO	Processed 30 items
2022-04-06 16:37:29,780	INFO	Done


,query_id,placekey,index,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,...,StreetNamePreDirectional,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,StreetNamePreType,PlaceName,StateName,AddressNumberSuffix
0,1,23m@8ts-zg6-ndv,1156,5401 e van buren st unit 1084,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,1084,NaN,NaN,NaN,NaN,NaN
1,2,22p@8ts-zg6-nnq,2327,5401 e van buren st unit 3109,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,3109,NaN,NaN,NaN,NaN,NaN
2,3,232@8ts-zg6-nnq,2354,5401 e van buren st unit 2101,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,2101,NaN,NaN,NaN,NaN,NaN
3,4,22z@8ts-zg6-ndv,2373,5401 e van buren st unit 3042,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,3042,NaN,NaN,NaN,NaN,NaN
4,5,252@8ts-zg6-hqz,2424,5302 e van buren st unit 1054,5302,e,van buren,st,None,unit,...,e,van buren,st,unit,1054,NaN,NaN,NaN,NaN,NaN
5,6,22t@8ts-zg6-hyv,2740,5345 e van buren st unit 325,5345,e,van buren,st,None,unit,...,e,van buren,st,unit,325,NaN,NaN,NaN,NaN,NaN
6,7,226@8ts-zg6-k4v,4071,5401 e van buren st unit 2120,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,2120,NaN,NaN,NaN,NaN,NaN
7,8,258@8ts-zg6-nh5,4077,5401 e van buren st unit 3022,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,3022,NaN,NaN,NaN,NaN,NaN


In [ ]:
# check original data
df_DATATREE[(df_DATATREE['SITUSSTREET']=='van buren')&(df_DATATREE['SITUSDIRECTIONLEFT']=='e')].head(6)
# ['DATATREE_ADDRESS'].str.contains('5401 e van buren')

,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,SITUSUNITNBR,SITUSLATITUDE,SITUSLONGITUDE,OWNERNAME1FULL,OWNERNAME2FULL,SITUSZIP5
69,3507 e van buren st,3507,e,van buren,st,None,None,None,33.450702,-112.005332,ustor-sky harbor llc,None,85008
984,4051 e van buren st,4051,e,van buren,st,None,None,None,33.450685,-111.993166,van buren land trust,None,85008
985,4119 e van buren st,4119,e,van buren,st,None,None,None,33.451206,-111.991249,masyno van buren company llc,None,85008
1167,5401 e van buren st unit 1084,5401,e,van buren,st,None,unit,1084,33.449058,-111.964291,jamal holdings us inc,None,85008
1409,2124 e van buren st,2124,e,van buren,st,None,None,None,33.451758,-112.035662,spirit realty lp,None,85006
2357,5401 e van buren st unit 3109,5401,e,van buren,st,None,unit,3109,33.449058,-111.964291,haehn jason,None,85008


### Question 3: Wanna change the parameters? 

In [ ]:
def get_placekey_df(pk_api,stad_df, place_keys:tuple):
    # stad_df is the df containing standatdized api fields
    import json
    # convert query_id to str
    stad_df = stad_df.astype({"query_id": str})
    
    # dump df to json-format with selected key-value pairs (only keep values in specified place_keys)
    data_jsoned = json.loads(stad_df.to_json(orient="records"))
    dict_filter = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])
    result_dict=[dict_filter(i, place_keys) for i in data_jsoned]
    
    #request placekey api
    responses = pk_api.lookup_placekeys(result_dict, 
                                        verbose=True,
                                        strict_address_match=False,
                                        strict_name_match=False)
    df_placekeys = pd.read_json(json.dumps(responses), dtype={'query_id':str}) 
    # TODO: need to handle missing / errors

    return df_placekeys

1. strict_address_match: \
If set to true, a Placekey is only returned if all fields identify the place as being at the exact address specified. If set to false, the Placekey returned may ignore unit/apartment/floor information => If strict_address_match is **false**, then address matches are fuzzy across **street_address, city, state, and postal_code**. If strict_address_match is **true**, then we **reject all matches that do not exactly match the input address fields**.

2. strict_name_match: (doesn't matter?)\
If set to true, a Placekey is only returned if all fields identify the POI as having the exact name specified => If strict_name_match is false, all location_name matches **after Step 1 are fuzzy**. If strict_name_match is true, all location_name matches are exact case-insensitive.

for more info: https://docs.placekey.io/#350ed3a9-68db-4c47-9e20-19b430cb9ef1

more info about their matching steps:

#### *Matching Behaviour*
Our matching algorithm tries a few different queries, sequentially, and returns the best match of the first query to return with high enough score to feel assured it's a true match. Here are the queries it does, in order:

1. If the address you've sent in is valid, then we search for a POI at that address placekey with a name that exactly case-insensitively matches the location_name you've sent in.
2. If this does not match (or if the address you sent in wasn't valid) but you've sent in a **latitude and longitude** with your query, then we search for that location_name and a fuzzy street address within 1km of your coordinates.
3. If this still does not match but you've sent in a postal code, then we search specifically for a POI in that **postal code** and look for a location_name match and a fuzzy street address match
4. If none of the above match and you have sent in a city and a region, then we require a strict match on city/region, a match on poi name, and a fuzzy match on street address.
5. Finally, if none of the above match, we stop searching for POI and perform an address match.

#### our interpretations: 
1) really doesn't matter? cuz in this step, we're looking for the comparision in varying unit number => the lat and long are the same!\
2) we've matched&generated keys based on the same zipcode => no need to worry about it

Q: so we only utilize their 1st step?

### Question 4: How to deal with incomplete unit info? - eg. when the unit signal is #, or it's missing

#### notes:
1. need to consider situations 1) when the address only has a unit signal without associated number, and 2) when there is no unit signal before the unit number 
2. **follow-up question**: if there is no unit signal, what should be put to signify the existence of unit?

In [ ]:
df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
pd.concat([df_placekey,test1.reset_index()],axis=1).head(6)


2022-04-06 16:26:56,924	INFO	Processed 30 items
2022-04-06 16:26:56,926	INFO	Done


,query_id,placekey,index,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,...,StreetNamePreDirectional,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,StreetNamePreType,PlaceName,StateName,AddressNumberSuffix
0,1,23m@8ts-zg6-ndv,1156,5401 e van buren st unit 1084,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,1084,NaN,NaN,NaN,NaN,NaN
1,2,22p@8ts-zg6-nnq,2327,5401 e van buren st unit 3109,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,3109,NaN,NaN,NaN,NaN,NaN
2,3,232@8ts-zg6-nnq,2354,5401 e van buren st unit 2101,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,2101,NaN,NaN,NaN,NaN,NaN
3,4,22z@8ts-zg6-ndv,2373,5401 e van buren st unit 3042,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,3042,NaN,NaN,NaN,NaN,NaN
4,5,252@8ts-zg6-hqz,2424,5302 e van buren st unit 1054,5302,e,van buren,st,None,unit,...,e,van buren,st,unit,1054,NaN,NaN,NaN,NaN,NaN
5,6,22t@8ts-zg6-hyv,2740,5345 e van buren st unit 325,5345,e,van buren,st,None,unit,...,e,van buren,st,unit,325,NaN,NaN,NaN,NaN,NaN


In [ ]:
import json
stad_df = stad_df.astype({"query_id": str})
# dump df to json-format with selected key-value pairs (only keep values in specified place_keys)
data_jsoned = json.loads(stad_df.to_json(orient="records"))
dict_filter = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])
result_dict=[dict_filter(i, place_keys) for i in data_jsoned]

In [ ]:
# output with "SITUSUNITTYPE" == "unit"
df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
pd.concat([df_placekey,test1.reset_index()],axis=1).head(6)

2022-04-06 17:03:46,900	INFO	Processed 30 items
2022-04-06 17:03:46,902	INFO	Done


,query_id,placekey,index,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,...,StreetNamePreDirectional,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,StreetNamePreType,PlaceName,StateName,AddressNumberSuffix
0,1,23m@8ts-zg6-ndv,1156,5401 e van buren st unit 1084,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,1084,NaN,NaN,NaN,NaN,NaN
1,2,22p@8ts-zg6-nnq,2327,5401 e van buren st unit 3109,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,3109,NaN,NaN,NaN,NaN,NaN
2,3,232@8ts-zg6-nnq,2354,5401 e van buren st unit 2101,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,2101,NaN,NaN,NaN,NaN,NaN
3,4,22z@8ts-zg6-ndv,2373,5401 e van buren st unit 3042,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,3042,NaN,NaN,NaN,NaN,NaN
4,5,252@8ts-zg6-hqz,2424,5302 e van buren st unit 1054,5302,e,van buren,st,None,unit,...,e,van buren,st,unit,1054,NaN,NaN,NaN,NaN,NaN
5,6,22t@8ts-zg6-hyv,2740,5345 e van buren st unit 325,5345,e,van buren,st,None,unit,...,e,van buren,st,unit,325,NaN,NaN,NaN,NaN,NaN


In [ ]:
# try to change the unit signal of the original data, see what happens
new_result_dict=[{'street_address': '5401 e van buren st apt 1084',
  'postal_code': '85008',
  'query_id': '1',
  'iso_country_code': 'US',
  'city': 'phoenix',
  'region': 'AZ'},
 {'street_address': '5401 e van buren st ste 3109',
  'postal_code': '85008',
  'query_id': '2',
  'iso_country_code': 'US',
  'city': 'phoenix',
  'region': 'AZ'},
 {'street_address': '5401 e van buren st ste 2101',
  'postal_code': '85008',
  'query_id': '3',
  'iso_country_code': 'US',
  'city': 'phoenix',
  'region': 'AZ'},
 {'street_address': '5401 e van buren st ste 3042',
  'postal_code': '85008',
  'query_id': '4',
  'iso_country_code': 'US',
  'city': 'phoenix',
  'region': 'AZ'},
 {'street_address': '5302 e van buren st ste 1054',
  'postal_code': '85008',
  'query_id': '5',
  'iso_country_code': 'US',
  'city': 'phoenix',
  'region': 'AZ'},
 {'street_address': '5345 e van buren st ste 325',
  'postal_code': '85008',
  'query_id': '6',
  'iso_country_code': 'US',
  'city': 'phoenix',
  'region': 'AZ'},
 {'street_address': '5401 e van buren st ste 2120',
  'postal_code': '85008',
  'query_id': '7',
  'iso_country_code': 'US',
  'city': 'phoenix',
  'region': 'AZ'}]

In [ ]:
# output with "SITUSUNITTYPE" != "unit"
responses = pk_api.lookup_placekeys(new_result_dict, 
                                    verbose=True,
                                    strict_address_match=False,
                                    strict_name_match=False)
df_placekeys = pd.read_json(json.dumps(responses), dtype={'query_id':str})
pd.concat([df_placekey,test1.reset_index()],axis=1).head(6)

2022-04-06 17:07:10,595	INFO	Processed 7 items
2022-04-06 17:07:10,597	INFO	Done


,query_id,placekey,index,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,...,StreetNamePreDirectional,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,StreetNamePreType,PlaceName,StateName,AddressNumberSuffix
0,1,23m@8ts-zg6-ndv,1156,5401 e van buren st unit 1084,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,1084,NaN,NaN,NaN,NaN,NaN
1,2,22p@8ts-zg6-nnq,2327,5401 e van buren st unit 3109,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,3109,NaN,NaN,NaN,NaN,NaN
2,3,232@8ts-zg6-nnq,2354,5401 e van buren st unit 2101,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,2101,NaN,NaN,NaN,NaN,NaN
3,4,22z@8ts-zg6-ndv,2373,5401 e van buren st unit 3042,5401,e,van buren,st,None,unit,...,e,van buren,st,unit,3042,NaN,NaN,NaN,NaN,NaN
4,5,252@8ts-zg6-hqz,2424,5302 e van buren st unit 1054,5302,e,van buren,st,None,unit,...,e,van buren,st,unit,1054,NaN,NaN,NaN,NaN,NaN
5,6,22t@8ts-zg6-hyv,2740,5345 e van buren st unit 325,5345,e,van buren,st,None,unit,...,e,van buren,st,unit,325,NaN,NaN,NaN,NaN,NaN


In [ ]:
DT_ADDRESS_final.SITUSUNITTYPE.unique()

array([None, 'apt', 'unit', 'ste', '#', 'lot'], dtype=object)

#### conlusion:
1. when we change unit types, the output placekeys are sill the same => so the varying unit types might not be a big issue (but we need to test with more data for sure!)
2. when "#" occues, maybe we can substitue it with the unit type occuring in the highest frequency? then we can create a new column with combined address info